In [ ]:
path = ''

# 환경설정

In [3]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import json





In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from warnings import filterwarnings
FutureWarning
filterwarnings('ignore')

c:\Users\User\anaconda3\envs\edusmile\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 환경 변수 설정 ( API KEY 설정)

In [ ]:
def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

# API 키 로드 및 환경변수 설정
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

In [ ]:
print(os.environ['OPENAI_API_KEY'])

# 음성

# 텍스트

In [ ]:
data = pd.read_csv(path + '/')
data.head()

#### 데이터 준비

In [ ]:
# 데이터셋 분할
train, val = train_test_split(data, test_size=0.2, random_state=42)
# 텐서 데이터셋으로 변환
train_ds = Dataset.from_pandas(train)
val_ds = Dataset.from_pandas(val)

#### (토큰화)

In [ ]:
# 모델과 토크나이저 불러오기
# 모델과 토크나이저 불러오기
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 토큰화 함수 생성 및 작업
def preprocess_function(data):
    return tokenizer(data['text'], truncation=True, padding=True)

train_ds = train_ds.map(preprocess_function, batched=True)
val_ds = val_ds.map(preprocess_function, batched=True)

#### 데이터 생성 (음성 추출)

In [25]:
import requests
import json
from pytube import YouTube
from pydub import AudioSegment

# Step 1: 유튜브 오디오 다운로드
def download_youtube_audio(video_url, output_path="audio/audio.wav"):
    # yt = YouTube(video_url)
    yt = YouTube(video_url, use_oauth=True, allow_oauth_cache=True)

    audio_stream = yt.streams.filter(only_audio=True).first()
    temp_audio_path = "audio/temp_audio.mp4"
    audio_stream.download(output_path="audio/", filename="temp_audio.mp4")
    
    # MP4 -> WAV 변환
    audio = AudioSegment.from_file(temp_audio_path, format="mp4")
    audio.export(output_path, format="wav")
    print("오디오 다운로드 및 변환 완료!")
    return output_path

# Step 2: 클로바 API를 사용한 음성 텍스트 변환
def audio_to_text_clova(audio_path, api_url, client_id, client_secret):
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/octet-stream"
    }

    with open(audio_path, "rb") as audio_file:
        response = requests.post(api_url, headers=headers, data=audio_file)
    
    if response.status_code == 200:
        result = response.json()
        return result.get("text", "")  # 변환된 텍스트 반환
    else:
        print(f"오류 발생: {response.status_code}, {response.text}")
        return None

# Step 3: 텍스트 저장
def save_text_to_file(text, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"텍스트가 {file_path}에 저장되었습니다!")



In [52]:
import os
import yt_dlp

def download_youtube_audio(video_url):
    try:
        # 지정된 절대 경로
        output_path = r"C:\Users\User\Desktop\eduSmile-master\audio"
        
        # 출력 디렉토리 생성
        os.makedirs(output_path, exist_ok=True)
        
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
            }],
            'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
            'ffmpeg_location': 'C:\\ffmpeg\\bin'  # ffmpeg 경로
        }
        
        print("다운로드 시작...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        print(f"다운로드 완료! 저장 위치: {output_path}")
        
    except Exception as e:
        print(f"Error: {str(e)}")



In [55]:
def process_youtube_urls():
    """
    txt 파일에서 YouTube URL을 읽어서 오디오를 다운로드하는 함수
    """
    # 파일 경로 설정
    file_path = r"C:\Users\User\Desktop\eduSmile-master\.git\datasets\youtube_datasets_01.txt"
    output_path = r"C:\Users\User\Desktop\eduSmile-master\audio"
    
    try:
        # 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as file:
            # https로 시작하는 URL만 필터링
            urls = [line.strip() for line in file if line.strip().startswith('https')]
        
        print(f"총 {len(urls)}개의 URL을 찾았습니다.")
        
        # 각 URL에 대해 오디오 다운로드
        for idx, url in enumerate(urls, 1):
            print(f"\n[{idx}/{len(urls)}] URL 처리 중...")
            download_youtube_audio(url)
            
    except FileNotFoundError:
        print(f"Error: 파일을 찾을 수 없습니다: {file_path}")
    except Exception as e:
        print(f"Error: 파일 처리 중 오류가 발생했습니다: {str(e)}")

In [56]:
process_youtube_urls()

Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


총 20개의 URL을 찾았습니다.

[1/20] URL 처리 중...
다운로드 시작...
[youtube] Extracting URL: https://www.youtube.com/watch?v=zlaMDt04gXY
[youtube] zlaMDt04gXY: Downloading webpage
[youtube] zlaMDt04gXY: Downloading ios player API JSON
[youtube] zlaMDt04gXY: Downloading mweb player API JSON


ERROR: [youtube] zlaMDt04gXY: Video unavailable
Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


Error: ERROR: [youtube] zlaMDt04gXY: Video unavailable

[2/20] URL 처리 중...
다운로드 시작...
[youtube] Extracting URL: https://www.youtube.com/watch?v=xBnq7xf2wt8
[youtube] xBnq7xf2wt8: Downloading webpage
[youtube] xBnq7xf2wt8: Downloading ios player API JSON
[youtube] xBnq7xf2wt8: Downloading mweb player API JSON
[youtube] xBnq7xf2wt8: Downloading player a5032d18


         player = https://www.youtube.com/s/player/a5032d18/player_ias.vflset/en_US/base.js
         n = WDsROWxyLpwasOWE ; player = https://www.youtube.com/s/player/a5032d18/player_ias.vflset/en_US/base.js
         player = https://www.youtube.com/s/player/a5032d18/player_ias.vflset/en_US/base.js
         n = RJKVHZKVCvX-oGpA ; player = https://www.youtube.com/s/player/a5032d18/player_ias.vflset/en_US/base.js


...

[youtube] E7YyNgT-qaY: Downloading m3u8 information
[info] E7YyNgT-qaY: Downloading 1 format(s): 251
[download] Destination: C:\Users\User\Desktop\eduSmile-master\audio\정수 사칙연산.webm
[download] 100% of    7.99MiB in 00:00:02 at 3.45MiB/s   
[ExtractAudio] Destination: C:\Users\User\Desktop\eduSmile-master\audio\정수 사칙연산.wav
Deleting original file C:\Users\User\Desktop\eduSmile-master\audio\정수 사칙연산.webm (pass -k to keep)


Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


다운로드 완료! 저장 위치: C:\Users\User\Desktop\eduSmile-master\audio

[20/20] URL 처리 중...
다운로드 시작...
[youtube] Extracting URL: https://www.youtube.com/watch?v=GnaQolCOvZw
[youtube] GnaQolCOvZw: Downloading webpage
[youtube] GnaQolCOvZw: Downloading ios player API JSON
[youtube] GnaQolCOvZw: Downloading mweb player API JSON


ERROR: [youtube] GnaQolCOvZw: Video unavailable


Error: ERROR: [youtube] GnaQolCOvZw: Video unavailable


#### 음성에서 텍스트로 변환

#### Whisper |

In [5]:
path = r'C:/Users/User/Desktop/eduSmile-master/'

In [10]:
def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

# API 키 로드 및 환경변수 설정
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

In [7]:
client = OpenAI()

In [24]:
audio_path = 'C:/Users/User/Desktop/eduSmile-master/audio/'  # 원본 오디오 파일이 저장된 폴더
chunk_path = 'C:/Users/User/Desktop/eduSmile-master/chunks/'  # 청크 파일 저장 폴더
output_path = 'C:/Users/User/Desktop/eduSmile-master/speech2text/'  # 변환된 텍스트 저장 폴더

In [26]:
import os
from pydub import AudioSegment
from openai import OpenAI

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


# 폴더 생성 함수
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# 필요한 폴더 생성
create_folder(chunk_path)
create_folder(output_path)

# 청크로 나누기
def split_audio(file_path, chunk_length_ms=10000):
    audio = AudioSegment.from_file(file_path)
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

    chunk_files = []
    for idx, chunk in enumerate(chunks):
        chunk_file = os.path.join(chunk_path, f"{file_name}_chunk_{idx}.wav")
        chunk.export(chunk_file, format="wav")
        chunk_files.append(chunk_file)

    return chunk_files

# 텍스트로 변환
def transcribe_audio(chunk_files):
    transcript_list = []
    for chunk_file in chunk_files:
        with open(chunk_file, "rb") as audio_file:
            try:
                transcript = client.audio.transcriptions.create(
                    file=audio_file,
                    model="whisper-1",
                    language="ko",
                    response_format="text"
                )
                # transcript는 Pydantic 모델 객체로 반환됨
                transcript_list.append(transcript)
            except Exception as e:
                print(f"Error processing {chunk_file}: {e}")
                transcript_list.append("")  # 오류 발생 시 빈 텍스트 추가
    return transcript_list

# 청크 텍스트 합치기 및 저장
def save_combined_text(transcripts, output_file):
    combined_text = "\n".join(transcripts)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(combined_text)

# 메인 실행
def process_audio_file(file_path):
    print(f"Processing file: {file_path}")
    chunk_files = split_audio(file_path)
    transcripts = transcribe_audio(chunk_files)
    
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    output_file = os.path.join(output_path, f"{file_name}_transcription.txt")
    save_combined_text(transcripts, output_file)
    print(f"Transcription saved to: {output_file}")

# 모든 오디오 파일 처리
for file_name in os.listdir(audio_path):
    if file_name.endswith(".wav"):  # .wav 파일만 처리
        process_audio_file(os.path.join(audio_path, file_name))


Processing file: C:/Users/User/Desktop/eduSmile-master/audio/0.wav
Transcription saved to: C:/Users/User/Desktop/eduSmile-master/speech2text/0_transcription.txt
Processing file: C:/Users/User/Desktop/eduSmile-master/audio/1.wav
Transcription saved to: C:/Users/User/Desktop/eduSmile-master/speech2text/1_transcription.txt
Processing file: C:/Users/User/Desktop/eduSmile-master/audio/10.wav
Transcription saved to: C:/Users/User/Desktop/eduSmile-master/speech2text/10_transcription.txt
Processing file: C:/Users/User/Desktop/eduSmile-master/audio/2.wav
Transcription saved to: C:/Users/User/Desktop/eduSmile-master/speech2text/2_transcription.txt
Processing file: C:/Users/User/Desktop/eduSmile-master/audio/3.wav
Transcription saved to: C:/Users/User/Desktop/eduSmile-master/speech2text/3_transcription.txt
Processing file: C:/Users/User/Desktop/eduSmile-master/audio/4.wav
Transcription saved to: C:/Users/User/Desktop/eduSmile-master/speech2text/4_transcription.txt
Processing file: C:/Users/User/D

#### 라벨링

In [ ]:
# Updated code to read input text from a specified file path and process it
import os
# Define the paths
input_file_path = os.path.join(path,"speech2text")
save_path = os.path.join(path,"ai_models/datasets/labels/")
os.makedirs(save_path, exist_ok=True)  # Create the directory if it doesn't exist


for file_name in os.listdir(input_file_path):
    file_path = os.path.join(input_file_path, file_name)
# Check if the input file exists
    if os.path.exists(file_path):
        # Read the text from the input file
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        # Split text into lines and group into 3-line blocks
        lines = text.split('\n')
        grouped_lines = [lines[i:i+3] for i in range(0, len(lines), 3)]

        # Function to label each group
        def label_group(group):
            group_text = ' '.join(group).strip()
            if any(keyword in group_text for keyword in ["정수", "수직선", "절대값"]):
                return 0 #"수업 내용"
            elif any(keyword in group_text for keyword in ["영상", "다음", "힘내시구요"]):
                return 1 #"공지사항"
            else:
                return 2 # "수업 내용과 관계 없음"

        # Label each group
        labeled_groups = [{"group": group, "label": label_group(group)} for group in grouped_lines]

        # Save labeled groups to a file
        output_file = os.path.join(save_path, "labeled_data.txt")
        with open(output_file, 'w', encoding='utf-8') as f:
            for entry in labeled_groups:
                f.write(f"Group:\n{' '.join(entry['group'])}\nLabel: {entry['label']}\n\n")

        print(f"Labeled data saved to: {output_file}")
    else:
        print("Input text file does not exist. Please ensure the file is at the specified path.")


In [29]:
import os
from sentence_transformers import SentenceTransformer, util

# Define the paths
input_file_path = os.path.join(path, "speech2text")
save_path = os.path.join(path, "ai_models/datasets/labels/")
os.makedirs(save_path, exist_ok=True)

# Initialize the Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Define category templates
categories = {
    0: "수업 내용: 곱셈, 나눗셈, 기호, 음수, 양수, 전진, 후퇴, 수직선, 절대값",
    1: "공지사항: 영상, 다음, 구독, 좋아요, 힘내시구요, 화이팅",
    2: "그 외 내용: 아기, 비유, 가족, 빌려주다, 뒷걸음질"
}

# Precompute embeddings for category templates
category_embeddings = {label: model.encode(text, convert_to_tensor=True) 
                      for label, text in categories.items()}

def chunk_text(text, chunk_size=300):
    """Split text into chunks of approximately chunk_size characters."""
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        word_length = len(word) + 1  # +1 for the space
        if current_length + word_length > chunk_size and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = word_length
        else:
            current_chunk.append(word)
            current_length += word_length
    
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

# Process each file in the directory
for file_num, file_name in enumerate(sorted(os.listdir(input_file_path)), 1):
    file_path = os.path.join(input_file_path, file_name)
    
    if os.path.exists(file_path):
        # Read the text from the input file
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        
        # Split text into 300-character chunks
        text_chunks = chunk_text(text, 300)
        labeled_chunks = []
        
        for index, chunk in enumerate(text_chunks):
            # Compute embedding for the chunk
            chunk_embedding = model.encode(chunk, convert_to_tensor=True)
            
            # Calculate cosine similarity with each category
            similarities = {
                label: util.cos_sim(chunk_embedding, category_embeddings[label]).item()
                for label in categories
            }
            
            # Determine the label with the highest similarity
            best_label = max(similarities, key=similarities.get)
            labeled_chunks.append({
                "chunk": chunk,
                "label": best_label,
                "index": index
            })
        
        # Save labeled chunks to a numbered file
        output_file = os.path.join(save_path, f"labeled_data_{file_num:02d}.txt")
        with open(output_file, 'w', encoding='utf-8') as f:
            for entry in sorted(labeled_chunks, key=lambda x: x['index']):
                f.write(f"Chunk {entry['index'] + 1} "
                       f"(Characters: {len(entry['chunk'])}):\n"
                       f"{entry['chunk']}\n"
                       f"Label: {entry['label']} ({categories[entry['label']]})\n\n")
        
        print(f"Labeled data saved to: {output_file}")
    else:
        print(f"Input file {file_name} does not exist.")

Labeled data saved to: C:/Users/User/Desktop/eduSmile-master/ai_models/datasets/labels/labeled_data_01.txt
Labeled data saved to: C:/Users/User/Desktop/eduSmile-master/ai_models/datasets/labels/labeled_data_02.txt
Labeled data saved to: C:/Users/User/Desktop/eduSmile-master/ai_models/datasets/labels/labeled_data_03.txt
Labeled data saved to: C:/Users/User/Desktop/eduSmile-master/ai_models/datasets/labels/labeled_data_04.txt
Labeled data saved to: C:/Users/User/Desktop/eduSmile-master/ai_models/datasets/labels/labeled_data_05.txt
Labeled data saved to: C:/Users/User/Desktop/eduSmile-master/ai_models/datasets/labels/labeled_data_06.txt
Labeled data saved to: C:/Users/User/Desktop/eduSmile-master/ai_models/datasets/labels/labeled_data_07.txt
Labeled data saved to: C:/Users/User/Desktop/eduSmile-master/ai_models/datasets/labels/labeled_data_08.txt
Labeled data saved to: C:/Users/User/Desktop/eduSmile-master/ai_models/datasets/labels/labeled_data_09.txt
Labeled data saved to: C:/Users/User/

#### 모델 학습

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import numpy as np

class LectureDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

def train_classifier(train_texts, train_labels, model_name='bert-base-multilingual-cased'):
    # 토크나이저와 모델 초기화
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2  # 수업내용(0)과 공지사항(1)
    )
    
    # 데이터셋 생성
    dataset = LectureDataset(train_texts, train_labels, tokenizer)
    train_loader = DataLoader(dataset, batch_size=8, shuffle=True)
    
    # 학습 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    
    # 학습 루프
    model.train()
    for epoch in range(3):  # 3 에폭 동안 학습
        for batch in train_loader:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            loss = outputs.loss
            loss.backward()
            optimizer.step()
    
    return model, tokenizer

def classify_notes(text, model, tokenizer, device='cuda'):
    """
    텍스트를 수업내용(0)과 공지사항(1)으로 분류
    """
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
    
    probabilities = torch.softmax(outputs.logits, dim=1)
    prediction = torch.argmax(probabilities, dim=1).item()
    
    return {
        'class': 'notice' if prediction == 1 else 'lecture',
        'probability': probabilities[0][prediction].item()
    }

# 사용 예시



In [ ]:
import openai
from typing import Dict, List
import os

class LectureNoteClassifier:
    def __init__(self, api_key: str):
        """
        OpenAI API를 사용하여 텍스트를 수업 내용, 공지사항, 관련없는 내용으로 분류하는 클래스
        
        Args:
            api_key (str): OpenAI API 키
        """
        openai.api_key = api_key
        
        # 분류를 위한 프롬프트 템플릿
        self.prompt_template = """다음 텍스트를 아래 세 카테고리 중 하나로 분류해주세요:
- lecture: 수업 내용 (예: 이론 설명, 개념 설명 등)
- notice: 수업 관련 공지사항 (예: 시험 일정, 과제 제출, 휴강 공지 등)
- unrelated: 수업과 관련없는 내용 (예: 개인적인 메모, 잡담, 다른 과목 내용 등)

답변은 위의 세 단어 중 하나만 작성해주세요.

텍스트: {text}

분류:"""

    def classify_line(self, text: str) -> Dict:
        """
        한 줄의 텍스트를 분류합니다.
        
        Args:
            text (str): 분류할 텍스트
            
        Returns:
            Dict: 분류 결과와 신뢰도를 포함한 딕셔너리
        """
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",  # 또는 "gpt-3.5-turbo"
                messages=[
                    {"role": "system", "content": "당신은 수업 노트를 수업 내용, 공지사항, 관련없는 내용으로 분류하는 도우미입니다."},
                    {"role": "user", "content": self.prompt_template.format(text=text)}
                ],
                temperature=0.3
            )
            
            # API 응답에서 분류 결과 추출
            classification = response.choices[0].message.content.strip().lower()
            
            return {
                'text': text,
                'class': classification,
                'probability': 1.0
            }
            
        except Exception as e:
            print(f"Error during classification: {e}")
            return {
                'text': text,
                'class': 'unknown',
                'probability': 0.0
            }

    def classify_notes(self, notes_text: str) -> Dict[str, List]:
        """
        여러 줄의 텍스트를 분류합니다.
        
        Args:
            notes_text (str): 분류할 전체 텍스트
            
        Returns:
            Dict: 수업 내용, 공지사항, 관련없는 내용으로 분류된 결과
        """
        # 줄바꿈을 기준으로 텍스트를 분리하고 빈 줄 제거
        lines = [line.strip() for line in notes_text.split('\n') if line.strip()]
        
        results = {
            'lecture_content': [],
            'notices': [],
            'unrelated_content': []
        }
        
        # 각 줄 분류
        for line in lines:
            classification = self.classify_line(line)
            if classification['class'] == 'lecture':
                results['lecture_content'].append({
                    'text': line,
                    'probability': classification['probability']
                })
            elif classification['class'] == 'notice':
                results['notices'].append({
                    'text': line,
                    'probability': classification['probability']
                })
            elif classification['class'] == 'unrelated':
                results['unrelated_content'].append({
                    'text': line,
                    'probability': classification['probability']
                })
        
        return results

# 사용 예시
def main():
    # OpenAI API 키 설정
    api_key = "your_openai_api_key_here"  # 실제 API 키로 교체 필요
    
    # 분류기 초기화
    classifier = LectureNoteClassifier(api_key)
    
    # 테스트용 노트
    test_notes = """오늘은 선형대수학의 기초를 배우겠습니다.
행렬의 곱셈은 다음과 같이 정의됩니다.
다음 주 퀴즈가 있으니 준비해주세요.
점심 메뉴 추천: 학식 오늘 맛있대.
벡터의 내적은 두 벡터의 성분끼리 곱한 것의 합입니다.
과제는 이번 주 금요일까지 제출해주세요.
오늘 날씨 진짜 덥다...
행렬식의 기하학적 의미는 면적 또는 부피를 나타냅니다.
저녁에 친구랑 약속 있음"""

    # 노트 분류
    results = classifier.classify_notes(test_notes)
    
    # 결과 출력
    print("\n=== 수업 내용 ===")
    for item in results['lecture_content']:
        print(f"• {item['text']}")
    
    print("\n=== 공지사항 ===")
    for item in results['notices']:
        print(f"• {item['text']}")
        
    print("\n=== 수업 무관 내용 ===")
    for item in results['unrelated_content']:
        print(f"• {item['text']}")

if __name__ == "__main__":
    main()

In [ ]:

# 학습 데이터 예시
train_texts = [
    "다음 주 시험은 3장부터 5장까지입니다.",  # 공지사항
    "오늘은 미분방정식의 기초에 대해 배우겠습니다.",  # 수업내용
    "과제 제출 기한은 이번 주 금요일까지입니다.",  # 공지사항
    "함수의 극한값을 구하는 방법은 다음과 같습니다."  # 수업내용
]
train_labels = [1, 0, 1, 0]  # 1: 공지사항, 0: 수업내용

# 모델 학습
model, tokenizer = train_classifier(train_texts, train_labels)

# 새로운 텍스트 분류
test_text = "다음 주는 중간고사 기간이므로 수업이 없습니다."
result = classify_notes(test_text, model, tokenizer)
print(f"분류 결과: {result['class']}, 확률: {result['probability']:.2f}")

#### 모델사용(claude)

In [53]:
# 1. 필요한 라이브러리 설치 (첫 번째 셀에서 실행)

# 2. 라이브러리 임포트와 클래스 정의 (두 번째 셀에서 실행)
import os
from datetime import datetime
from anthropic import Anthropic
import json
from pathlib import Path
import time
from typing import Dict, List, Any

class ClaudeProcessor:
    def __init__(self, api_key: str, input_folder: str, output_folder: str = "수업자료"):
        """
        Claude API를 사용하여 수업 내용을 처리하는 클래스
        """
        self.input_folder = Path(input_folder)
        self.output_folder = Path(output_folder)
        self.client = Anthropic(api_key=api_key)
        self._create_folders()
        self.last_api_call = 0
        self.min_delay = 1
    
    def _create_folders(self):
        """필요한 폴더들을 생성합니다."""
        folders = ['공지사항', '수업내용', '문제', '해설']
        for folder in folders:
            folder_path = self.output_folder / folder
            folder_path.mkdir(parents=True, exist_ok=True)
    
    def _generate_filename(self, prefix: str, original_filename: str) -> str:
        """원본 파일명을 포함한 결과 파일명을 생성합니다."""
        date_str = datetime.now().strftime('%Y%m%d_%H%M%S')
        original_name = Path(original_filename).stem
        return f"{prefix}_{original_name}_{date_str}"
    
    def _save_text(self, filepath: Path, content: str):
        """텍스트 내용을 파일로 저장합니다."""
        with open(f"{filepath}.txt", 'w', encoding='utf-8') as f:
            f.write(content)
    
    def _save_json(self, filepath: Path, content: Dict):
        """JSON 내용을 파일로 저장합니다."""
        with open(f"{filepath}.json", 'w', encoding='utf-8') as f:
            json.dump(content, f, ensure_ascii=False, indent=2)
    
    def _read_file(self, file_path: Path) -> str:
        """파일 내용을 읽습니다."""
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    
    def _respect_rate_limit(self):
        """API 호출 간격을 조절합니다."""
        current_time = time.time()
        time_since_last_call = current_time - self.last_api_call
        if time_since_last_call < self.min_delay:
            time.sleep(self.min_delay - time_since_last_call)
        self.last_api_call = time.time()
    
    def get_claude_response(self, prompt: str) -> str:
        """Claude API를 호출하여 응답을 받습니다."""
        self._respect_rate_limit()
        try:
            response = self.client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=2000,
                temperature=0.7,
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]
            )
            if isinstance(response.content, list):
                if len(response.content) > 0 and hasattr(response.content[0], 'text'):
                    return response.content[0].text
                return ' '.join(str(item) for item in response.content)
            elif hasattr(response.content, 'text'):
                return response.content.text
            return str(response.content)
        except Exception as e:
            print(f"API 호출 중 오류 발생: {str(e)}")
            raise

    def generate_quiz(self, content: str, num_questions: int = 5) -> Dict:
        """객관식 문제를 생성합니다."""
        prompt = f"""
        다음 수업 내용을 바탕으로 {num_questions}개의 객관식 문제를 생성해주세요.
        반드시 아래의 JSON 형식만을 사용하여 응답해주세요.
        추가 설명이나 다른 텍스트는 포함하지 마세요.

        {{
            "questions": [
                {{
                    "question": "문제 내용",
                    "choices": ["보기1", "보기2", "보기3", "보기4"],
                    "correct_answer": 0,
                    "explanation": "해설"
                }}
            ]
        }}

        수업 내용:
        {content}
        """
        
        try:
            response_text = self.get_claude_response(prompt)
            json_start = response_text.find('{')
            json_end = response_text.rfind('}') + 1
            
            if json_start == -1 or json_end == 0:
                raise ValueError("JSON 형식을 찾을 수 없습니다")
                
            json_str = response_text[json_start:json_end]
            quiz = json.loads(json_str)
            
            if not isinstance(quiz, dict) or 'questions' not in quiz:
                raise ValueError("잘못된 JSON 구조")
                
            for q in quiz['questions']:
                if not all(key in q for key in ['question', 'choices', 'correct_answer', 'explanation']):
                    raise ValueError("문제 형식이 잘못되었습니다")
                if not isinstance(q['choices'], list) or len(q['choices']) != 4:
                    raise ValueError("보기 형식이 잘못되었습니다")
                if not isinstance(q['correct_answer'], (int, float)) or not (0 <= int(q['correct_answer']) <= 3):
                    q['correct_answer'] = 0
            
            return quiz
            
        except json.JSONDecodeError as e:
            print(f"JSON 파싱 오류: {str(e)}")
            print(f"응답 텍스트: {response_text}")
        except Exception as e:
            print(f"문제 생성 중 오류 발생: {str(e)}")
        
        return {
            "questions": [
                {
                    "question": "문제를 생성하는 중 오류가 발생했습니다.",
                    "choices": ["다시 시도해주세요"] * 4,
                    "correct_answer": 0,
                    "explanation": "문제 생성에 실패했습니다."
                }
            ]
        }

    def process_file(self, file_path: Path) -> Dict:
        """개별 파일의 내용을 처리합니다."""
        content = self._read_file(file_path)
        filename = file_path.name
        results = {}
        
        try:
            # 공지사항 추출
            notice = self.get_claude_response(f"""
            다음 수업 내용에서 공지사항만을 추출해주세요. 
            공지사항이 없다면 "공지사항이 없습니다."라고 답변해주세요.
            
            수업 내용:
            {content}
            """)
            notice_path = self.output_folder / '공지사항' / self._generate_filename('공지', filename)
            self._save_text(notice_path, notice)
            results['공지사항'] = {'내용': notice, '저장경로': f"{notice_path}.txt"}
            
            # 수업내용 요약
            summary = self.get_claude_response(f"""
            다음 수업 내용을 핵심 개념과 중요 포인트 위주로 요약해주세요.
            각 개념에 대해 구체적인 예시를 포함해주세요.
            
            수업 내용:
            {content}
            """)
            summary_path = self.output_folder / '수업내용' / self._generate_filename('수업요약', filename)
            self._save_text(summary_path, summary)
            results['수업요약'] = {'내용': summary, '저장경로': f"{summary_path}.txt"}
            
            # 객관식 문제 생성
            quiz = self.generate_quiz(content)
            quiz_path = self.output_folder / '문제' / self._generate_filename('객관식문제', filename)
            self._save_json(quiz_path, quiz)
            results['객관식문제'] = {'내용': quiz, '저장경로': f"{quiz_path}.json"}
            
            # 해설 생성
            explanations = {
                "문제해설": [
                    {
                        "문제번호": i + 1,
                        "문제": q["question"],
                        "정답": f"{int(q['correct_answer']) + 1}번",
                        "해설": q["explanation"]
                    }
                    for i, q in enumerate(quiz["questions"])
                ]
            }
            explanations_path = self.output_folder / '해설' / self._generate_filename('문제해설', filename)
            self._save_json(explanations_path, explanations)
            results['문제해설'] = {'내용': explanations, '저장경로': f"{explanations_path}.json"}
            
        except Exception as e:
            print(f"파일 처리 중 오류 발생: {filename}")
            print(f"오류 내용: {str(e)}")
            results['오류'] = {'내용': str(e), '파일명': filename}
        
        return results

    def process_all_files(self) -> Dict[str, Dict]:
        """모든 파일을 처리합니다."""
        all_results = {}
        supported_extensions = {'.txt', '.md', '.text'}
        
        for file_path in self.input_folder.iterdir():
            if file_path.suffix.lower() in supported_extensions:
                print(f"처리 중: {file_path.name}")
                try:
                    results = self.process_file(file_path)
                    all_results[file_path.name] = results
                    print(f"처리 완료: {file_path.name}")
                except Exception as e:
                    print(f"파일 처리 중 오류 발생: {file_path.name}")
                    print(f"오류 내용: {str(e)}")
                    all_results[file_path.name] = {'오류': {'내용': str(e)}}
        
        return all_results

def process_folder_contents(api_key: str, input_folder: str, output_folder: str = "수업자료") -> Dict:
    """폴더 내의 모든 수업 자료를 처리하는 편의 함수"""
    processor = ClaudeProcessor(api_key, input_folder, output_folder)
    return processor.process_all_files()

def display_results(all_results: Dict):
    """처리 결과를 보기 좋게 출력합니다."""
    for filename, results in all_results.items():
        print(f"\n{'='*50}")
        print(f"파일: {filename}")
        print('='*50)
        
        if '오류' in results:
            print(f"오류 발생: {results['오류']['내용']}")
            continue
            
        for category, data in results.items():
            print(f"\n=== {category} ===")
            print(f"저장경로: {data['저장경로']}")
            print("\n내용:")
            
            if category == '객관식문제':
                for i, question in enumerate(data['내용']['questions'], 1):
                    print(f"\n{i}번 문제: {question['question']}")
                    for j, choice in enumerate(question['choices'], 1):
                        print(f"   {j}) {choice}")
                    print(f"   정답: {int(question['correct_answer']) + 1}번")
                    print(f"   해설: {question['explanation']}")
            elif category == '문제해설':
                for explanation in data['내용']['문제해설']:
                    print(f"\n{explanation['문제번호']}번 문제")
                    print(f"문제: {explanation['문제']}")
                    print(f"정답: {explanation['정답']}")
                    print(f"해설: {explanation['해설']}")
            else:
                print(data['내용'])



In [54]:
api_key = ''
input_folder = "C:/Users/User/Desktop/eduSmile-master/speech2text"  # 수업 내용 파일이 있는 폴더
output_folder = "C:/Users/User/Desktop/eduSmile-master/results"  # 결과물을 저장할 폴더

In [55]:
 # 처리 실행
try:
    results = process_folder_contents(api_key, input_folder, output_folder)
    
    # 결과 출력
    display_results(results)
except Exception as e:
    print(f"처리 중 오류가 발생했습니다: {str(e)}")

처리 중: 0_transcription.txt
처리 완료: 0_transcription.txt
처리 중: 10_transcription.txt
처리 완료: 10_transcription.txt
처리 중: 1_transcription.txt
문제 생성 중 오류 발생: 보기 형식이 잘못되었습니다
처리 완료: 1_transcription.txt
처리 중: 2_transcription.txt
처리 완료: 2_transcription.txt
처리 중: 3_transcription.txt
처리 완료: 3_transcription.txt
처리 중: 4_transcription.txt
처리 완료: 4_transcription.txt
처리 중: 5_transcription.txt
처리 완료: 5_transcription.txt
처리 중: 6_transcription.txt
처리 완료: 6_transcription.txt
처리 중: 7_transcription.txt
처리 완료: 7_transcription.txt
처리 중: 8_transcription.txt
처리 완료: 8_transcription.txt
처리 중: 9_transcription.txt
처리 완료: 9_transcription.txt

파일: 0_transcription.txt

=== 공지사항 ===
저장경로: C:\Users\User\Desktop\eduSmile-master\results\공지사항\공지_0_transcription_20250110_114654.txt

내용:
이 수업 내용에는 공지사항이 없습니다.

=== 수업요약 ===
저장경로: C:\Users\User\Desktop\eduSmile-master\results\수업내용\수업요약_0_transcription_20250110_114659.txt

내용:
이번 수업의 핵심 개념과 중요 포인트는 다음과 같습니다:

1. 정수의 덧셈 (수직선 활용)
   - 양의 정수 + 양의 정수 = 양의 정수의 합
   - 양의 정수 + 음의 정수 = 양의 